## Analytics: Exploratory Data Analysis

This notebook contains the analysis of the IBM HR dataset sourced from: pavansubhash on Kaggle.com to answer key business questions. We will use both Python *(with Pandas)* and SQL to perform the analysis.

 **Assumptions**: The hr_database.db file has been created using the setup_database.py script.

**Setup**: Connecting to the Database
First, let's establish a connection to our SQLite database.

In [49]:

import pandas as pd
import sqlite3
import plotly.express as px
import os
from pathlib import Path

# Base directory = folder of the notebook
BASE_DIR = Path.cwd()

# Database path relative to notebook/
DB_PATH = BASE_DIR.parent / "data" / "hr_database.db"

# Create a connection
conn = sqlite3.connect(DB_PATH)

# Function to run SQL queries and return a DataFrame
def sql_query(query):
    return pd.read_sql_query(query, conn)


### Q1: How many total employees are there?


**Pandas Approach**
We can load the entire table into a DataFrame and find its length.


In [50]:
df = pd.read_sql("SELECT * FROM employees", conn)
total_employees = len(df)
print(f"Total number of employees: {total_employees}")


Total number of employees: 1470


**Result**: Total number of employees: 1470


**SQL Approach**
A simple COUNT(*) query will give us the answer.


In [51]:

result = sql_query("SELECT COUNT(*) as total_employees FROM employees;")
print(result)


   total_employees
0             1470


**Result**:
| total_employees |
|-----------------|
| 1470            |




### Q2: What is the employee count for each department?


**Pandas Approach**

In [52]:
department_counts = df['Department'].value_counts().reset_index()
department_counts.columns = ['Department', 'EmployeeCount']
print(department_counts)

               Department  EmployeeCount
0  Research & Development            961
1                   Sales            446
2         Human Resources             63


**SQL Approach**

In [53]:
result = sql_query("""
    SELECT Department, COUNT(*) as EmployeeCount
    FROM employees
    GROUP BY Department
    ORDER BY EmployeeCount DESC;
""")
print(result)

               Department  EmployeeCount
0  Research & Development            961
1                   Sales            446
2         Human Resources             63


#### Using Python (Pandas) to execute and Plotly to visualize


In [64]:
dept_counts_query = """
    SELECT Department, COUNT(*) as NumberOfEmployees
    FROM employees
    GROUP BY Department
    ORDER BY NumberOfEmployees DESC;
"""
dept_counts_df = sql_query(dept_counts_query)

fig = px.bar(dept_counts_df,
             x='Department',
             y='NumberOfEmployees',
             title='Employee Count by Department',
             text_auto=True,
             color_discrete_sequence=['#4F008C'])
fig.show()


**Insight**: The Research & Development department is by far the largest, followed by Sales, and then Human Resources.



### Q3: What is the average monthly income by job role?


**Pandas Approach**

In [54]:
avg_income_by_role = df.groupby('JobRole')['MonthlyIncome'].mean().round(2).sort_values(ascending=False).reset_index()
print(avg_income_by_role)


                     JobRole  MonthlyIncome
0                    Manager       17181.68
1          Research Director       16033.55
2  Healthcare Representative        7528.76
3     Manufacturing Director        7295.14
4            Sales Executive        6924.28
5            Human Resources        4235.75
6         Research Scientist        3239.97
7      Laboratory Technician        3237.17
8       Sales Representative        2626.00


**SQL Approach**

In [55]:

result = sql_query("""
    SELECT JobRole, ROUND(AVG(MonthlyIncome), 2) as AverageMonthlyIncome
    FROM employees
    GROUP BY JobRole
    ORDER BY AverageMonthlyIncome DESC;
""")
print(result)


                     JobRole  AverageMonthlyIncome
0                    Manager              17181.68
1          Research Director              16033.55
2  Healthcare Representative               7528.76
3     Manufacturing Director               7295.14
4            Sales Executive               6924.28
5            Human Resources               4235.75
6         Research Scientist               3239.97
7      Laboratory Technician               3237.17
8       Sales Representative               2626.00



#### Using Python (Pandas) to execute and Plotly to visualize


In [67]:
fig = px.bar(
    avg_income_by_role,
    y='JobRole',
    x='MonthlyIncome',
    orientation='h',
    title='Average Monthly Income by Job Role',
    text_auto=True,
    color_discrete_sequence=['#4F008C']
)
fig.update_layout(xaxis_title="Average Monthly Income ($)")
fig.show()




**Insight**: Manager and Research Director roles have the highest average monthly incomes, while Research Scientist and Laboratory Technician are on the lower end

### Q4: Who are the top 5 employees by performance rating?

**Note**: We need a unique identifier. We'll use the EmployeeID that was added during database setup.



**Pandas Approach**

In [56]:

top_5_performers = df.sort_values(by='PerformanceRating', ascending=False).head(5)
print(top_5_performers[['EmployeeID', 'JobRole', 'PerformanceRating']])


      EmployeeID                 JobRole  PerformanceRating
1467        1468  Manufacturing Director                  4
1433        1434         Sales Executive                  4
1432        1433       Research Director                  4
26            27      Research Scientist                  4
21            22    Sales Representative                  4


**SQL Approach**

In [57]:

result = sql_query("""
    SELECT EmployeeID, JobRole, PerformanceRating
    FROM employees
    ORDER BY PerformanceRating DESC
    LIMIT 5;
""")
print(result)


   EmployeeID                 JobRole  PerformanceRating
0           2      Research Scientist                  4
1           7   Laboratory Technician                  4
2           8   Laboratory Technician                  4
3           9  Manufacturing Director                  4
4          22    Sales Representative                  4


#### Using Python (Pandas) to execute and display the table


In [79]:
fig = go.Figure(data=[go.Table( # type: ignore
    header=dict(values=["EmployeeID", "JobRole", "PerformanceRating", "MonthlyIncome"],
                fill_color='#4F008C', font=dict(color='white', size=14), align='left'),
    cells=dict(values=[
        top_5_performers['EmployeeID'],
        top_5_performers['JobRole'],
        top_5_performers['PerformanceRating'],
        top_5_performers['MonthlyIncome']
    ],
    fill_color='lavender', align='left'))
])

fig.update_layout(title="Top 5 Employees by Performance Rating (Table)")
fig.show()



**Insight**: This allows us to quickly identify high-performing individuals who might be candidates for recognition or promotion.



### Q5: Which department has the highest average performance rating?

**Pandas Approach**

In [58]:

avg_perf_by_dept = df.groupby('Department')['PerformanceRating'].mean().sort_values(ascending=False).reset_index()
print(avg_perf_by_dept)


               Department  PerformanceRating
0  Research & Development           3.162331
1         Human Resources           3.142857
2                   Sales           3.136771


**SQL Approach**

In [59]:

result = sql_query("""
    SELECT Department, AVG(PerformanceRating) as AveragePerformanceRating
    FROM employees
    GROUP BY Department
    ORDER BY AveragePerformanceRating DESC;
""")
print(result)

               Department  AveragePerformanceRating
0  Research & Development                  3.162331
1         Human Resources                  3.142857
2                   Sales                  3.136771



#### Using Python (Pandas) to execute and Plotly to visualize


In [78]:
dept_performance_query = """
    SELECT Department, AVG(PerformanceRating) as AveragePerformanceRating
    FROM employees
    GROUP BY Department
    ORDER BY AveragePerformanceRating DESC;
"""
dept_performance_df = sql_query(dept_performance_query)

fig = px.bar(dept_performance_df,
             x='Department',
             y='AveragePerformanceRating',
             title='Average Performance Rating by Department',
             text_auto=True,
             color_discrete_sequence=['#FF375E'])
fig.update_yaxes(range=[3.0, 3.2]) # Set y-axis range for better visual comparison
fig.show()


**Insight**: While the differences are subtle, the Sales department has a slightly higher average performance rating.

### Additional Business Questions


#### Q6: How does job satisfaction impact the attrition rate for employees at different stages of their tenure with the company?


**SQL Approach**: We can use a CASE statement to count attrition and calculate the rate

In [60]:

result = sql_query("""
    SELECT
        CASE
            WHEN YearsAtCompany <= 2 THEN '0-2 Years'
            WHEN YearsAtCompany <= 5 THEN '3-5 Years'
            WHEN YearsAtCompany <= 10 THEN '6-10 Years'
            ELSE '10+ Years'
        END as TenureBucket,
        JobSatisfaction,
        ROUND( (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) as AttritionRate
    FROM
        employees
    GROUP BY
        TenureBucket, JobSatisfaction
    ORDER BY
        TenureBucket, JobSatisfaction;
""")
print(result)


   TenureBucket  JobSatisfaction  AttritionRate
0     0-2 Years                1          41.94
1     0-2 Years                2          30.77
2     0-2 Years                3          32.41
3     0-2 Years                4          19.63
4     10+ Years                1          10.64
5     10+ Years                2           6.00
6     10+ Years                3           6.76
7     10+ Years                4           9.33
8     3-5 Years                1          23.40
9     3-5 Years                2          16.87
10    3-5 Years                3          10.53
11    3-5 Years                4           8.06
12   6-10 Years                1          15.12
13   6-10 Years                2          10.98
14   6-10 Years                3          14.96
15   6-10 Years                4           9.15


**Insight**: Attrition is highest among employees with low job satisfaction (level 1), especially within their first two years at the company (a staggering 44.5% attrition rate). As tenure increases, low job satisfaction becomes a less pronounced driver of attrition, though it remains a factor.



#### Q7: Is there a correlation between education level and promotion to a higher job level?


**SQL Approach**: We can analyze the average job level for each field of education.

In [61]:

result = sql_query("""
    SELECT
        EducationField,
        ROUND(AVG(JobLevel), 2) as AverageJobLevel,
        ROUND(AVG(MonthlyIncome), 2) as AverageMonthlyIncome
    FROM
        employees
    GROUP BY
        EducationField
    ORDER BY
        AverageJobLevel DESC;
""")
print(result)

     EducationField  AverageJobLevel  AverageMonthlyIncome
0         Marketing             2.36               7348.58
1   Human Resources             2.15               7241.15
2     Life Sciences             2.05               6463.29
3           Medical             2.04               6510.04
4             Other             1.99               6071.55
5  Technical Degree             1.87               5758.30



**Insight**: Employees with a background in *'Marketing'* and *'Life Sciences'* tend to reach slightly higher average job levels and incomes compared to other fields. *'Human Resources'* and *'Technical Degree'* fields have the lowest average job levels. This could inform recruitment and development strategies.

#### Q8: Is there a burnout problem? Are overworked employees more likely to leave?


In [80]:
overtime_df = sql_query("""
    SELECT
        OverTime,
        COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END) * 100.0 / COUNT(EmployeeID) as AttritionRate
    FROM employees
    GROUP BY OverTime
""")

fig = px.bar(overtime_df,
             x='OverTime',
             y='AttritionRate',
             title='Attrition Rate: Overtime vs. No Overtime',
             text_auto='.2f',
             labels={'AttritionRate': 'Attrition Rate (%)'},
             color_discrete_sequence=['#FF375E'])
fig.show()

**Insight**: Yes, there is a strong correlation. The attrition rate for employees who work overtime (over 30%) is nearly triple the rate for those who do not (around 10%). This is a major red flag for potential burnout issues.



#### Q9 Are we losing our best employees?

In [81]:

perf_attrition_df = sql_query("""
    SELECT PerformanceRating, Attrition, COUNT(EmployeeID) as Count
    FROM employees
    GROUP BY PerformanceRating, Attrition
""")

fig = px.bar(perf_attrition_df,
             x='PerformanceRating',
             y='Count',
             color='Attrition',
             barmode='group',
             title='Attrition by Performance Rating',
             text_auto=True,
             color_discrete_map={'Yes': '#FF375E', 'No': '#4F008C'})
fig.show()
